In [ ]:
import dtreeviz
import pandas as pd
import numpy as np
from sklearn.datasets import *
from sklearn import tree


In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from itertools import cycle
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
from scipy import interp
import catboost
import xgboost
import shap
from sklearn.preprocessing import label_binarize
from catboost import *
from sklearn.metrics import roc_curve, auc

%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Ionization Potential',#0
                      'Electronegativity',#1
                      'Number of d electrons',#2
                      'ZIF or MOF Derived',#3
                      'Polymer Derived',#4
                      'Carbon Nanofiber/Nanotubes',#5
                      'Biomass or other Organic Derived',#6  
                      'Main Transition Metal Content (wt. %)',#7
                      'Nitrogen Cotent (wt. %)',#8
                      'Metal-N Coordination Number (XAS)',#9    
                      'Pyridinic N Ratio',#10
                      'Pyrrolic N Ratio',#11
                      'Raman ID/IG Ratio',#12
                      'BET Surface Area (m2/g)',#13
                      'Pyrolysis Temperature (°C)',#14
                      'Pyrolysis Time (h)',#15
                      'Rising Rate (°C min-1)',#16
                      'Electrolyte Concentration (M)',#17
                      'Catalyst Loading (mg cm-2)',#18
                      'Nafion Membrane Thickness (μm)',#19
                      'Carbon Paper/Glassy Carbon',#20
                      'Electrolyte pH',#21
                      'ovpCL',#22
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:22]
raw_power=raw_data.iloc[:,22]
print('ready')
nb_classes=3

In [ ]:
from catboost import *
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    Y_test_labeled = label_binarize(y_test, classes=[i for i in range(nb_classes)])
    y_score=best_model.predict_proba(X_test)
#     y_score=y_score[:,1]
#     print(y_score)
    auc_curve(Y_test_labeled,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    return best_model
def auc_curve(y_label, y_pre,algorithm_name):
#     y_label = y_label + 1
#     y_pre = y_pre + 1
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(nb_classes):
        fpr[i], tpr[i], _ = roc_curve(y_label[:, i], y_pre[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_label.ravel(), y_pre.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(nb_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(nb_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= nb_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])



    # Plot all ROC curves
    lw = 2
    fig=plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(nb_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Multi-Class Classification) of %s' %algorithm_name)
    plt.legend(loc="lower right")
    plt.savefig('ROC Curve of %s OVPMCL.png' %algorithm_name)
def shap_plot_multi(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:22]
    SHAP_OUTPUT=raw_data.iloc[:,22]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1]), SHAP_INPUT,max_display=100)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
def shap_plot_0(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:22]
    SHAP_OUTPUT=raw_data.iloc[:,22]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
   
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[0]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[0], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
def shap_plot_1(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:22]
    SHAP_OUTPUT=raw_data.iloc[:,22]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[1]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[1]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[1]).mean(0)
        print(global_importances)
def shap_plot_2(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:22]
    SHAP_OUTPUT=raw_data.iloc[:,22]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
   
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        original_shape = shap_values.shape
        shap_values_reshaped = shap_values.reshape(original_shape[1], original_shape[0], original_shape[-1])
        shap.summary_plot(list(shap_values_reshaped[:,:,:-1])[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(list(shap_values_reshaped[:,:,:-1])[2]).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[2]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[2], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[2]).mean(0)
        print(global_importances)


In [ ]:
seed=184
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.1,random_state=seed)



In [ ]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
        'max_depth':[8],
        'criterion' : ['entropy'],
        'splitter' : [ "best"],
        'max_features':['auto']
}
DT_OVPMCL=gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')

In [ ]:
from dtreeviz.trees import dtreeviz
viz = dtreeviz(DT_OVPMCL,
               raw_param,
               raw_power,
               target_name='',
               feature_names=raw_param.columns,
               class_names={0:'<=0.5V',1:'0.5~0.8V',2:'>=0.8V'},scale=2)
              
viz

In [ ]:
type(viz)

In [ ]:
viz.view()